In [ ]:
!pip3 install vngrs-nlp

In [ ]:
pip install -U sentence-transformers

In [3]:
pip install JPype1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 3.2 MB/s eta 0:00:00


In [6]:
import pandas as pd
import numpy as np
import zeyrek
import nltk##lemmatization için lazım 
from jpype import JClass, getDefaultJVMPath, shutdownJVM, startJVM, JString
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import string
from sklearn.ensemble import RandomForestRegressor
import re
from transformers import AutoTokenizer, AutoModel
import torch
from sentence_transformers import SentenceTransformer
from vnlp import DependencyParser,PoSTagger,StopwordRemover, StemmerAnalyzer
#from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
#from transformers import AutoTokenizer, AutoModel
#import torch
#from vnlp import DependencyParser,PoSTagger,StopwordRemover, StemmerAnalyzer

In [ ]:
df1 = pd.read_csv("./data_result_added_mine", sep='\t')
df2 = pd.read_csv("./data_result_added", sep='\t')
frames = [df1, df2]
data = pd.concat(frames,ignore_index=True)


In [ ]:
from vnlp import DependencyParser,PoSTagger,StopwordRemover, StemmerAnalyzer

In [ ]:
translator = str.maketrans('', '', string.punctuation)
nltk.download('punkt')
stopword_remover = StopwordRemover()
analyzer = zeyrek.MorphAnalyzer()

In [83]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

tokenizer = AutoTokenizer.from_pretrained('emrecan/bert-base-turkish-cased-mean-nli-stsb-tr')
model = AutoModel.from_pretrained('emrecan/bert-base-turkish-cased-mean-nli-stsb-tr')
def giveEmbeddings(sentences):
  encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')
  with torch.no_grad():
      model_output = model(**encoded_input)
  sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])
  return sentence_embeddings


In [ ]:
def title_similarity():
    data["result"]=0.0
    for i in range(len(data.sentence)):
        title=stopword_remover.drop_stop_words(nltk.word_tokenize(data.loc[i,"title"].translate(translator).replace("“","").replace("’","")))
        embeddings=giveEmbeddings([data.loc[i,"sentence"],data.loc[i,"title"]])
        similarity=cosine_similarity([embeddings[0].numpy() ], [embeddings[1].numpy()])
        data.loc[i,"result"]=similarity[0][0]


In [ ]:
title_similarity()

In [ ]:
import pandas as pd

In [ ]:
data.to_csv("last_data",sep="\t",index=False)


In [28]:
data = pd.read_csv("./last_data", sep='\t')

In [ ]:
print(np.sum(data))

news_id                                                     18299693.0
sentence             Ancak 1990’larından sonundan itibaren Türkiye,...
title                Dünyanın yükselen değeri TürkiyeDünyanın yükse...
is_contains_idiom                                               -18891
NER                                                              24706
tfidf                                                             6957
noun_phrase                                                      12854
words2               ['Ancak', '1990', '’', 'larından', 'sonundan',...
sentence_length                                            1064.322524
position                                                   3287.272428
result                                                     4310.908189
dtype: object


In [ ]:
print(len(data))

18891


In [29]:
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.svm import SVR
import pickle
import math
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
import inspect

In [32]:
data=data.dropna()
data.isnull().sum()
data.reset_index(drop=True, inplace=True)

In [33]:
import random

# Initialize an empty set to store unique numbers
unique_numbers = set()
uniques=data.news_id.unique() 
# Generate random numbers until we have 2000 unique numbers
while len(unique_numbers) < 852:
    # Generate a random number between 0 and 2500
    random_number = random.randint(0, len(uniques))
    # Add the number to the set if it's not already in it
    unique_numbers.add(uniques[random_number])

# Convert the set to a list and sort it
unique_numbers_list = sorted(list(unique_numbers))

In [34]:
X_rand_train=[]
X_rand_test=[]
for i in range(len(data)):
  if data.loc[i,"news_id"] in unique_numbers_list:
    X_rand_train.append(data.loc[i,:])
  else:
    X_rand_test.append(data.loc[i,:])

ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/usr/local/lib/python3.10/dist-packages/keras/backend.py", line 5159, in <genexpr>
    output_ta_t = tuple(  File "/usr/local/lib/python3.10/dist-packages/tensorflow/python/util/tf_should_use.py", line 243, in wrapped
    return _add_should_use_warning(fn(*args, **kwargs),


In [35]:
len(X_rand_train)

15023

In [ ]:
len(X_rand_test)

3077

In [37]:
random_train = pd.DataFrame (X_rand_train, columns = data.columns)
random_test = pd.DataFrame (X_rand_test, columns = data.columns)

In [38]:
X_tr=random_train.loc[:,["news_id","sentence","title","NER","noun_phrase","tfidf","sentence_length","position"]]
X_ts=random_test.loc[:,["news_id","sentence","title","NER","noun_phrase","tfidf","sentence_length","position"]]
y_train=random_train.loc[:,["result"]]
y_test=random_test.loc[:,["result"]]

In [39]:
X_train=X_tr.loc[:,["NER","noun_phrase","tfidf","sentence_length","position"]]
X_test=X_ts.loc[:,["NER","noun_phrase","tfidf","sentence_length","position"]]

In [40]:
regressor = RandomForestRegressor(n_estimators = 600, random_state = 42)
regressor.fit(X_train,y_train) 

<ipython-input-40-9f03bb6509aa>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  regressor.fit(X_train,y_train)


RandomForestRegressor(n_estimators=600, random_state=42)

In [41]:
Y_pred = regressor.predict(X_test)

In [42]:
m1=mean_squared_error(y_test,Y_pred)
m2=mean_absolute_error(y_test,Y_pred)
r2=r2_score(y_test,Y_pred, force_finite=False)
print(m1,m2,r2)

0.025612581471014806 0.12385581608334119 0.1326944346824661


In [ ]:
clf = linear_model.Lasso(alpha=0.5)
clf.fit(X_train,y_train)

Lasso(alpha=0.5)

In [ ]:
y_pred2=clf.predict(X_test)

In [ ]:
mL1=mean_squared_error(y_test,y_pred2)
mL2=mean_absolute_error(y_test,y_pred2)
r2L=r2_score(y_test,y_pred2, force_finite=False)
print(mL1,mL2,r2L)

0.03095196612669707 0.14241248082987648 -0.0017881752125712413


In [ ]:
logr  = LinearRegression()
logr.fit(X_train,y_train) 

LinearRegression()

In [ ]:
y_pred3= logr.predict(X_test)
mL1=mean_squared_error(y_test,y_pred3)
mL2=mean_absolute_error(y_test,y_pred3)
r2L=r2_score(y_test,y_pred3, force_finite=False)
print(mL1,mL2,r2L)

0.024609451720946855 0.12365472521370946 0.181542494741878


,is_contains_idiom,NER,noun_phrase,tfidf,sentence_length,position
70,-1,4,0,0,0.128205,1.000000
71,-1,2,3,0,0.076923,0.500000
72,-1,0,0,0,0.068376,0.333333
73,-1,1,1,0,0.068376,0.250000
74,-1,2,1,0,0.085470,0.200000
...,...,...,...,...,...,...
11355,-1,1,0,0,0.031414,0.076923
11356,-1,3,0,0,0.070681,0.071429
11357,-1,1,0,1,0.052356,0.066667
11358,-1,0,0,0,0.028796,0.062500


In [43]:
X_ts.reset_index(drop=True, inplace=True)
X_ts

,news_id,sentence,title,NER,noun_phrase,tfidf,sentence_length,position
0,149.0,Erzincan'da dindar vatandaşlara yönelik hukuks...,"Gül, Erdoğan ve Arınça ağır hakaret",2,1,0,0.041667,1.000000
1,149.0,Ordu Komutanı Saldıray Berk'in ilginç bir icra...,"Gül, Erdoğan ve Arınça ağır hakaret",1,1,0,0.013889,0.500000
2,149.0,Kara Kuvvetleri Komutanlığına ait özel ve dışa...,"Gül, Erdoğan ve Arınça ağır hakaret",2,0,1,0.029321,0.333333
3,149.0,Orgeneral Saldıray Berk komutanlığındaki 3.,"Gül, Erdoğan ve Arınça ağır hakaret",1,0,0,0.007716,0.250000
4,149.0,"Ordu Komutanlığının intranet sitesinde, Cumhur...","Gül, Erdoğan ve Arınça ağır hakaret",5,3,0,0.038580,0.200000
...,...,...,...,...,...,...,...,...
3863,1697.0,Antrenman böylece eğlenceli bir şekilde sona e...,Floryada Lucas Neill rüzgarı,1,1,0,0.032864,0.111111
3864,1697.0,KEWELL VE NONDA RİSKE SOKULMADI Sarı-kırmızılı...,Floryada Lucas Neill rüzgarı,2,0,0,0.122066,0.100000
3865,1697.0,"İki futbolcu, üst sahada koşu yapmakla yetinir...",Floryada Lucas Neill rüzgarı,1,0,1,0.084507,0.090909
3866,1697.0,"Kewell'ın sakatlığının düzeldiği, Nonda'nın du...",Floryada Lucas Neill rüzgarı,1,0,0,0.056338,0.083333


In [44]:
def getMaxs(X,Y,Y_real):
  Y_real=Y_real.to_numpy()
  max_values = []
  real_max_values=[]
  index=-1
  real_index=-1
  curr_id=0
  max=-1
  real_max=-1
  for i in range(len(X.loc[:,"news_id"])):
    if curr_id != X.loc[i,"news_id"].astype(int):
      curr_id=X.loc[i,"news_id"].astype(int)
      if index != -1:
        max_values.append(X.loc[index,:])
        real_max_values.append(X.loc[real_index,:])
      max=-1
      index=-1
      real_max=-1
      real_index=-1
    if Y[i]>max:
      max=Y[i]
      index=i
    if Y_real[i][0]>real_max:
      real_max=Y_real[i][0]
      real_index=i
  return (max_values,real_max_values)
    


In [88]:
(a,r_a)=getMaxs(X_ts,Y_pred,y_test)

In [50]:
a=a[:10]

In [51]:
summed=0
for i in range(len(a)):
  if a[i]["sentence"]==r_a[i]["sentence"]:
    summed+=1

In [52]:
summed/len(a)

0.3

In [53]:
dep_parser = DependencyParser()
def dependency_parser():
    deps=[]
    i=0
    
    for sentence in a:
        #print(sentence)
        try:
            parsed=dep_parser.predict(sentence["sentence"])
            deps.append(parsed)
        except Exception:
          try:
            h=sentence["sentence"].split()
            parsed=dep_parser.predict(" ".join(h[0:len(h)//2]))
            parsed+=dep_parser.predict(" ".join(h[len(h)//2:len(h)]))
            deps.append(parsed)
          except Exception:
            deps.append(sentence["sentence"])
            continue
        i+=1
    return deps

In [54]:
depsss=dependency_parser()

In [55]:
depsss

[[(1, "Erzincan'da", 16, 'obl'),
  (2, 'dindar', 3, 'amod'),
  (3, 'vatandaşlara', 6, 'nmod'),
  (4, 'yönelik', 3, 'case'),
  (5, 'hukuksuz', 6, 'amod'),
  (6, 'gözaltılar', 7, 'nmod:poss'),
  (7, 'sebebiyle', 16, 'obl'),
  (8, 'savcılığın', 9, 'nmod:poss'),
  (9, 'ifadesini', 10, 'obj'),
  (10, 'almak', 11, 'xcomp'),
  (11, 'istemesine', 15, 'advcl'),
  (12, 'rağmen', 11, 'case'),
  (13, 'günlerdir', 15, 'advmod'),
  (14, 'ifadeye', 15, 'obl'),
  (15, 'gitmeyen', 0, 'root'),
  (16, ';', 28, 'punct'),
  (17, 'bu', 19, 'nsubj'),
  (18, 'da', 17, 'advmod:emph'),
  (19, 'yetmezmiş', 28, 'advcl'),
  (20, 'gibi', 19, 'case'),
  (21, 'önceki', 22, 'amod'),
  (22, 'gün', 28, 'obl'),
  (23, 'de', 22, 'advmod:emph'),
  (24, 'askeri', 25, 'amod'),
  (25, 'araçları', 27, 'obj'),
  (26, 'sokağa', 27, 'obl'),
  (27, 'döken', 26, 'acl'),
  (28, '3', 17, 'nummod'),
  (29, '.', 28, 'punct')],
 [(1, 'Adalet', 2, 'nmod:poss'),
  (2, 'Bakanlığı', 22, 'nsubj'),
  (3, ',', 2, 'punct'),
  (4, 'askerlere', 8

In [56]:
import pickle
with open("test4", "wb") as fp:   #Pickling
  pickle.dump(depsss, fp)


In [58]:
titles=[]
for deps_pars in depsss:
  root=0
  zero=0
  title = set()
  indexes=set()
  deps_pars
  if type(deps_pars[0])==str:
    titles.append("şok şok şok")
    continue
  if len(deps_pars)<14:
    titles.append("")
    continue
  for j in range(len(deps_pars)):
    i=deps_pars[j]
    if i[2]==0:
      zero=i
      indexes.add(i[0])
      title.add(i)
    if i[3]=="root":
      root=i
      indexes.add(i[0])
      title.add(i)
  
  for j in range(len(deps_pars)):
    i=deps_pars[j]
    if (root!=0 and i[0]+1==root[0]) or (zero!=0 and  i[0]+1==zero[0]):
      if i[2]!="punct":
        title.add(i)
        indexes.add(i[0])

    if  (root!=0 and (i[0]-1==root[0] or i[0]+1==root[0]) and i[3]=="conj") or (zero!=0 and (i[0]-1==zero[0] or i[0]+1==zero[0]) and i[3]=="conj"):
      if i[2]!="punct":
        title.add(i)
        indexes.add(i[0])

    if ((root!=0 and i[2]==root[0] ) or (zero!=0 and i[2]==zero[0]) ) and (i[3]=="nsubj" or i[3]=="compound" or i[3]=="flat"   ):
        if i[2]!="punct":
          title.add(i)
          indexes.add(i[0])

  for i in deps_pars:
    if i[2] in indexes and (i[3]=="flat" or i[3]=="amod" or i[3]=="nmod" or i[3]=="nmod:poss" or i[3]=="csubj" or i[3]=="obl" or i[3]=="compound" or i[3]=="det" or i[3]=="nummod"):
      if i[2]!="punct":
        title.add(i)
        indexes.add(i[0])

  for i in deps_pars:
    if i[2] in indexes and (i[3]=="flat" or i[3]=="amod" or i[3]=="nmod" or i[3]=="nmod:poss" or i[3]=="csubj" or i[3]=="obl" or i[3]=="compound" or i[3]=="det" or i[3]=="nummod"):
      if i[2]!="punct":
        title.add(i)
        indexes.add(i[0])

  for i in deps_pars:
    if i[2] in indexes and (i[3]=="flat" or i[3]=="amod" or i[3]=="nmod" or i[3]=="nmod:poss" or i[3]=="csubj" or i[3]=="obl" or i[3]=="compound" or i[3]=="det" or i[3]=="nummod"):
      if i[2]!="punct":
        title.add(i)
        indexes.add(i[0])
  if(len(title)==0 or title==None):
    title="şok şok şok"
  titles.append(title)




In [59]:
basliks=[]
for title in titles:
  baslik=""
  if type(title)==str:
    basliks.append(title)
    continue
  for kelime in title:
    baslik +=" "+kelime[1]
  basliks.append(baslik)



In [60]:
basliks

[' gitmeyen ifadeye',
 ' taslağı Bakanlığı iptalinden kanun bir hazırladı yasanın teklifi Adalet',
 ' olduğunu vurguladı',
 " aldığından Cem emin kendinden Ahmet Cübbeli Hoca Yılmaz'ın",
 ' " başta araçlarda dedi Aydın ÇİN DİKKAT parçaları MALINA',
 ' ortaya olduğu çıkmış',
 ' balans sözlerini demokrasiye yapılması ayarı hatırlattı',
 ' yeniden üniversiteye Merkezi değiştirecek',
 ' çarptı )',
 '']

In [65]:
a

[news_id                                                        149.0
 sentence           Erzincan'da dindar vatandaşlara yönelik hukuks...
 title                            Gül, Erdoğan ve Arınça ağır hakaret
 NER                                                                2
 noun_phrase                                                        1
 tfidf                                                              0
 sentence_length                                             0.041667
 position                                                         1.0
 Name: 0, dtype: object,
 news_id                                                        150.0
 sentence           Adalet Bakanlığı, askerlere sivil yargı yolunu...
 title                              Hükümet Askerin peşini bırakmıyor
 NER                                                                1
 noun_phrase                                                        0
 tfidf                                                           

In [66]:
from summa import summarizer
print()

NameError: ignored

In [ ]:
with open("test", "rb") as fp:   # Unpickling
  b = pickle.load(fp)

In [ ]:
raw_news = pd.read_csv("./sample2500.csv", sep='\t')


In [63]:
!pip install summa
  

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.9/54.9 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for summa: filename=summa-1.2.0-py3-none-any.whl size=54408 sha256=2dac14c5c8ce05b728a3ec05191dffdd8db4d8bbdacd0528137e654b658dc6c7
  Stored in directory: /root/.cache/pip/wheels/4a/ca/c5/4958614cfba88ed6ceb7cb5a849f9f89f9ac49971616bc919f
Successfully built summa


In [74]:
guess=

SyntaxError: ignored

news_id                                                        149.0
sentence           Erzincan'da dindar vatandaşlara yönelik hukuks...
title                            Gül, Erdoğan ve Arınça ağır hakaret
NER                                                                2
noun_phrase                                                        1
tfidf                                                              0
sentence_length                                             0.041667
position                                                         1.0
Name: 0, dtype: object
news_id                                                        150.0
sentence           Adalet Bakanlığı, askerlere sivil yargı yolunu...
title                              Hükümet Askerin peşini bırakmıyor
NER                                                                1
noun_phrase                                                        0
tfidf                                                              1
sentence_le

In [73]:
predicted_s

['', '', '', '', '', '', '', '', '', '']

In [70]:
a

[news_id                                                        149.0
 sentence           Erzincan'da dindar vatandaşlara yönelik hukuks...
 title                            Gül, Erdoğan ve Arınça ağır hakaret
 NER                                                                2
 noun_phrase                                                        1
 tfidf                                                              0
 sentence_length                                             0.041667
 position                                                         1.0
 Name: 0, dtype: object,
 news_id                                                        150.0
 sentence           Adalet Bakanlığı, askerlere sivil yargı yolunu...
 title                              Hükümet Askerin peşini bırakmıyor
 NER                                                                1
 noun_phrase                                                        0
 tfidf                                                           

In [76]:
g = pd.read_csv("./sample2500.csv", sep='\t')

In [77]:
g

,text,title
0,Araç sahipleri için pahalı benzine karşı yapıl...,Aracınızda yüzde 30 tarassuf edin
1,"İki insanın kaderini birleştiren olay, Adana S...",Kahraman öğretmen hayat kurtardı
2,Meksika'nın kuzey sınırında bir grup silahlı k...,Parti yapanları taradılar: 13 ölü
3,"Fenerbahçe Teknik Direktörü Christoph Daum, Si...",Daum oynanan oyundan memnun
4,İslam Konferansı Teşkilatı (İKT) Daimi Konsey ...,İslam Konferansında minare tepkisi
...,...,...
2496,"Time dergisinin foto muhabiri Shaul Schwarz, k...",Kızgın Haitililer cesetten barikat kurdu
2497,Hacı Bektaş Veli Anadolu Kültür Vakfı (HBVAKV)...,Aleviler yeni sol partide yok
2498,Muayene için gittiği Almanya'da ameliyat olan ...,Galatasarayda Milan Baros şoku!
2499,Kızıl gezegenin etrafında 2006'dan beri dönmek...,Mars fotoğraflarındaki ağacın sırrı


In [79]:
a=[]
a.append(g.loc[149,"text"])
a.append(g.loc[150,"text"])
a.append(g.loc[154,"text"])
a.append(g.loc[159,"text"])
a.append(g.loc[175,"text"])
a.append(g.loc[178,"text"])
a.append(g.loc[187,"text"])
a.append(g.loc[199,"text"])
a.append(g.loc[108,"text"])
a.append(g.loc[220,"text"])
a

['Erzincan\'da dindar vatandaşlara yönelik hukuksuz gözaltılar sebebiyle savcılığın ifadesini almak istemesine rağmen günlerdir ifadeye gitmeyen; bu da yetmezmiş gibi önceki gün de askeri araçları sokağa döken 3. Ordu Komutanı Saldıray Berk\'in ilginç bir icraatı tespit edildi. Kara Kuvvetleri Komutanlığına ait özel ve dışarıdan girilemeyen özel bir intranet ağı olan KaraNET\'te açıkça siyaset yapıldığı ortaya çıktı. Orgeneral Saldıray Berk komutanlığındaki 3. Ordu Komutanlığının intranet sitesinde, Cumhurbaşkanı Abdullah Gül, Başbakan Recep Tayyip Erdoğan ile Başbakan Yardımcısı Bülent Arınç hakkında ağır ifadeler içeren yazılar yer aldığı tespit edildi. Söz konusu yazılar komutanlık sitesine "Mutlaka okunması gerekenler" başlığı altında toplanmış. 3. Ordu Komutanı Saldıray Berk\'in talimatıyla hazırlandığı belirtilen sitedeki "Mutlaka okunması gerekenler" başlığı altında, Cumhuriyet gazetesi başta olmak üzere ART\'de çalışan birçok Ergenekoncunun yazıları yer alıyor. Özellikle Ak Par

In [81]:
predicted_s=[]
for ss in a:
  print(ss)
  predicted_s.append(summarizer.summarize(ss))

Erzincan'da dindar vatandaşlara yönelik hukuksuz gözaltılar sebebiyle savcılığın ifadesini almak istemesine rağmen günlerdir ifadeye gitmeyen; bu da yetmezmiş gibi önceki gün de askeri araçları sokağa döken 3. Ordu Komutanı Saldıray Berk'in ilginç bir icraatı tespit edildi. Kara Kuvvetleri Komutanlığına ait özel ve dışarıdan girilemeyen özel bir intranet ağı olan KaraNET'te açıkça siyaset yapıldığı ortaya çıktı. Orgeneral Saldıray Berk komutanlığındaki 3. Ordu Komutanlığının intranet sitesinde, Cumhurbaşkanı Abdullah Gül, Başbakan Recep Tayyip Erdoğan ile Başbakan Yardımcısı Bülent Arınç hakkında ağır ifadeler içeren yazılar yer aldığı tespit edildi. Söz konusu yazılar komutanlık sitesine "Mutlaka okunması gerekenler" başlığı altında toplanmış. 3. Ordu Komutanı Saldıray Berk'in talimatıyla hazırlandığı belirtilen sitedeki "Mutlaka okunması gerekenler" başlığı altında, Cumhuriyet gazetesi başta olmak üzere ART'de çalışan birçok Ergenekoncunun yazıları yer alıyor. Özellikle Ak Parti ile 

In [82]:
predicted_s

['Ordu Komutanı Saldıray Berk\'in ilginç bir icraatı tespit edildi.\nKara Kuvvetleri Komutanlığına ait özel ve dışarıdan girilemeyen özel bir intranet ağı olan KaraNET\'te açıkça siyaset yapıldığı ortaya çıktı.\nOrdu Komutanlığının intranet sitesinde, Cumhurbaşkanı Abdullah Gül, Başbakan Recep Tayyip Erdoğan ile Başbakan Yardımcısı Bülent Arınç hakkında ağır ifadeler içeren yazılar yer aldığı tespit edildi.\n3. Ordu Komutanı Saldıray Berk\'in talimatıyla hazırlandığı belirtilen sitedeki "Mutlaka okunması gerekenler" başlığı altında, Cumhuriyet gazetesi başta olmak üzere ART\'de çalışan birçok Ergenekoncunun yazıları yer alıyor.\n- Er ve erbaşlara kesinlikle yasak olan Kara Kuvvetleri Komutanlığına ait özel bir intranet sistemidir.\nİki kişinin ölümü ve yüzlercesinin yaralanmasıyla sonuçlanan bu olaylar siyasal tarihimize "Kanlı Pazar" olarak geçerken olayların olduğu saatlerde gerici bir grup Boğaz\'a demirlemiş Amerikan 6.\nOrdu Komutanı Org. Saldıray Berk\'in talimatıyla hazırlanan s

In [84]:
def title_similarity():
    for i in range(len(data.sentence)):
        title=stopword_remover.drop_stop_words(nltk.word_tokenize(data.loc[i,"title"].translate(translator).replace("“","").replace("’","")))
        embeddings=giveEmbeddings([data.loc[i,"sentence"],data.loc[i,"title"]])
        similarity=cosine_similarity([embeddings[0].numpy() ], [embeddings[1].numpy()])
        data.loc[i,"result"]=similarity[0][0]

In [92]:
toplam=0
for f in range(10):
  embeddings=giveEmbeddings([basliks[f],predicted_s[f]])
  similarity=cosine_similarity([embeddings[0].numpy()], [embeddings[1].numpy()])
  print(similarity)     

In [93]:
toplam/10

array([[0.31325048]], dtype=float32)

In [ ]:
toplam=0
for f in range(10):
  embeddings=giveEmbeddings([a[f]["title"],predicted_s[f]])
  similarity=cosine_similarity([embeddings[0].numpy()], [embeddings[1].numpy()])
  toplam+=  similarity  